In [29]:
import pandas as pd

# Load tabular dataset
table = pd.read_csv('input_table.csv',header=0,index_col=0)     # <-- Your main data

# (Optional) Load QA dataset (for testing, if available)
qa_data = pd.read_excel('QA_dataset_share.xlsx')        # <-- Questions & answers

# Show preview
print("Table Preview:")
print(table.head())

print("\nQA Dataset Preview:")
print(qa_data.head())


Table Preview:
                  City  Gender            Product line  Unit price  Quantity  \
Invoice ID                                                                     
750-67-8428     Yangon  Female       Health and beauty       74.69         7   
226-31-3081  Naypyitaw  Female  Electronic accessories       15.28         5   
631-41-3108     Yangon    Male      Home and lifestyle       46.33         7   
123-19-1176     Yangon    Male       Health and beauty       58.22         8   
373-73-7910     Yangon    Male       Sports and travel       86.31         7   

                Total       Date      Payment  gross income  Rating  
Invoice ID                                                           
750-67-8428  548.9715   1/5/2019      Ewallet       26.1415     9.1  
226-31-3081   80.2200   3/8/2019         Cash        3.8200     9.6  
631-41-3108  340.5255   3/3/2019  Credit card       16.2155     7.4  
123-19-1176  489.0480  1/27/2019      Ewallet       23.2880     8.4  
373-

In [ ]:
!pip install sentence-transformers

In [5]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
# After restarting the kernel, run this cell:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')  # free, fast, small

def get_relevant_rows_and_cols(query, df, top_n=5):
    # Flatten table text for column-level matching
    column_texts = df.columns.tolist()
    row_texts = df.astype(str).apply(lambda x: ' '.join(x), axis=1).tolist()

    # Embed everything
    query_emb = model.encode(query, convert_to_tensor=True)
    row_embs = model.encode(row_texts, convert_to_tensor=True)
    col_embs = model.encode(column_texts, convert_to_tensor=True)

    # Compute similarity
    row_scores = util.pytorch_cos_sim(query_emb, row_embs)[0]
    col_scores = util.pytorch_cos_sim(query_emb, col_embs)[0]

    # Get top N
    top_row_indices = row_scores.topk(top_n).indices.tolist()
    top_col_indices = col_scores.topk(top_n).indices.tolist()

    return top_row_indices, top_col_indices

In [7]:
!pip install --upgrade sentence-transformers transformers


In [8]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')  # free, fast, small

def get_relevant_rows_and_cols(query, df, top_n=5):
    # Flatten table text for column-level matching
    column_texts = df.columns.tolist()
    row_texts = df.astype(str).apply(lambda x: ' '.join(x), axis=1).tolist()

    # Embed everything
    query_emb = model.encode(query, convert_to_tensor=True)
    row_embs = model.encode(row_texts, convert_to_tensor=True)
    col_embs = model.encode(column_texts, convert_to_tensor=True)

    # Compute similarity
    row_scores = util.pytorch_cos_sim(query_emb, row_embs)[0]
    col_scores = util.pytorch_cos_sim(query_emb, col_embs)[0]

    # Get top N
    top_row_indices = row_scores.topk(top_n).indices.tolist()
    top_col_indices = col_scores.topk(top_n).indices.tolist()

    return top_row_indices, top_col_indices


In [24]:
from transformers import pipeline
import pandas as pd
import torch

# Check if GPU is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise CPU

# Create QA pipeline with explicit device placement
qa_pipeline = pipeline(
    "question-answering", 
    model="distilbert-base-uncased-distilled-squad",
    device=device
)

def generate_answer(query, df, row_indices=None, col_indices=None, max_context_length=512):
    """
    Generate answers to questions based on DataFrame content.
    
    Args:
        query (str): The question to answer
        df (pd.DataFrame): Source DataFrame containing the data
        row_indices (list, optional): Specific rows to include. If None, use all rows.
        col_indices (list, optional): Specific columns to include. If None, use all columns.
        max_context_length (int, optional): Maximum context length to avoid token limits
        
    Returns:
        dict: Full result dictionary with answer, score, and positions
    """
    # Handle default indices
    row_indices = row_indices if row_indices is not None else range(len(df))
    col_indices = col_indices if col_indices is not None else df.columns
    
    # Extract context (filtered data)
    filtered_df = df.iloc[row_indices, col_indices]
    context = filtered_df.to_string(index=False)
    
    # Truncate context if too long (to avoid token limits)
    if len(context) > max_context_length:
        context = context[:max_context_length]
    
    # Ask question
    result = qa_pipeline(question=query, context=context)
    
    # Return full result for more information
    return result


Device set to use cpu


In [17]:
import openpyxl

def generate_output(query, df):
    rows, cols = get_relevant_rows_and_cols(query, df)
    answer = generate_answer(query, df, rows, cols)

    output_df = pd.DataFrame({
        'filtered row index': [rows],
        'filtered column index': [cols],
        'generated response': [answer]
    })

    output_df.to_excel('predicted.xlsx', index=False)
    return output_df



In [18]:
print("✅ Step 1: Table Loaded")
print("✅ Step 2: Retriever Output:", rows, cols)
print("✅ Step 3: Generated Response:", answer)
print("✅ Step 4: Output saved to predicted.xlsx")


✅ Step 1: Table Loaded


NameError: name 'rows' is not defined

In [21]:
# Import pandas library first
import pandas as pd

# Define the table variable
table = pd.read_csv('input_table.csv')  # Replace this with your actual table data
# For example:
# table = pd.read_csv('your_data.csv')
# OR
# table = pd.DataFrame({'Category': ['Health and Beauty', 'Electronics', ...],
#                      'Gross Income': [1500, 2000, ...]})

query = "What is the highest gross income for health and beauty?"
rows, cols = get_relevant_rows_and_cols(query, table)
answer = generate_answer(query, table, rows, cols)

print("Rows:", rows)
print("Cols:", cols)
print("Answer:", answer)


Rows: [749, 141, 426, 523, 14]
Cols: [9, 6, 10, 2, 5]
Answer: {'score': 0.02001830004155636, 'start': 102, 'end': 143, 'answer': '45.2500 950.2500     8.1   Male        10'}


In [22]:
# Import pandas library first
import pandas as pd

# Define the table variable
table = pd.read_csv('input_table.csv')  # Replace this with your actual table data
# For example:
# table = pd.read_csv('your_data.csv')
# OR
# table = pd.DataFrame({'Category': ['Health and Beauty', 'Electronics', ...],
#                      'Gross Income': [1500, 2000, ...]})

query = "What is highest rating?"
rows, cols = get_relevant_rows_and_cols(query, table)
answer = generate_answer(query, table, rows, cols)

print("Rows:", rows)
print("Cols:", cols)
print("Answer:", answer)


Rows: [312, 727, 590, 426, 589]
Cols: [10, 6, 2, 4, 5]
Answer: {'score': 0.13640296459197998, 'start': 50, 'end': 69, 'answer': '4.4  74.7075 Female'}


In [23]:
# Import pandas library first
import pandas as pd

# Define the table variable
table = pd.read_csv('input_table.csv')  # Replace this with your actual table data
# For example:
# table = pd.read_csv('your_data.csv')
# OR
# table = pd.DataFrame({'Category': ['Health and Beauty', 'Electronics', ...],
#                      'Gross Income': [1500, 2000, ...]})

query = "what is the max rating given in home and lifestyle?"
rows, cols = get_relevant_rows_and_cols(query, table)
answer = generate_answer(query, table, rows, cols)

print("Rows:", rows)
print("Cols:", cols)
print("Answer:", answer)


Rows: [540, 416, 253, 268, 22]
Cols: [10, 6, 9, 4, 5]
Answer: {'score': 0.0526006706058979, 'start': 56, 'end': 59, 'answer': '9.1'}


In [30]:
# Import pandas library first
import pandas as pd

# Define the table variable
table = pd.read_csv('input_table.csv')  # Replace this with your actual table data
# For example:
# table = pd.read_csv('your_data.csv')
# OR
# table = pd.DataFrame({'Category': ['Health and Beauty', 'Electronics', ...],
#                      'Gross Income': [1500, 2000, ...]})

query = "What product line is in the latest entry?"
rows, cols = get_relevant_rows_and_cols(query, table)
answer = generate_answer(query, table, rows, cols)

print("Rows:", rows)
print("Cols:", cols)
print("Answer:", answer)


Rows: [12, 811, 782, 655, 549]
Cols: [3, 7, 4, 5, 6]
Answer: {'score': 0.7685032486915588, 'start': 64, 'end': 86, 'answer': 'Electronic accessories'}


In [35]:
# Import pandas library first
import pandas as pd

# Define the table variable
table = pd.read_csv('input_table.csv',index_col=0)  # Replace this with your actual table data
# For example:
# table = pd.read_csv('your_data.csv')
# OR
# table = pd.DataFrame({'Category': ['Health and Beauty', 'Electronics', ...],
#                      'Gross Income': [1500, 2000, ...]})

query = "What product line is in the latest entry?"
rows, cols = get_relevant_rows_and_cols(query, table)
answer = generate_answer(query, table, rows, cols)

print("Rows:", rows)
print("Cols:", cols)
print("Answer:", answer)

Rows: [655, 369, 303, 674, 549]
Cols: [2, 6, 3, 4, 5]
Answer: {'score': 0.8057325482368469, 'start': 64, 'end': 86, 'answer': 'Electronic accessories'}


In [34]:
# Import pandas library first
import pandas as pd

# Define the table variable
table = pd.read_csv('input_table.csv')  # Replace this with your actual table data
# For example:
# table = pd.read_csv('your_data.csv')
# OR
# table = pd.DataFrame({'Category': ['Health and Beauty', 'Electronics', ...],
#                      'Gross Income': [1500, 2000, ...]})

query = "which product  is most sold in mandalay as per quantity?"
rows, cols = get_relevant_rows_and_cols(query, table)
answer = generate_answer(query, table, rows, cols)

print("Rows:", rows)
print("Cols:", cols)
print("Answer:", answer)

Rows: [174, 28, 734, 638, 724]
Cols: [5, 3, 4, 6, 7]
Answer: {'score': 0.6045435070991516, 'start': 70, 'end': 88, 'answer': 'Food and beverages'}
